## La Chasse au Wumpus

*La chasse au Wumpus* est un célèbre jeu d'IA dans lequel un personnage, nommé Alan dans ce TP, doit trouver un trésor dans un labyrinthe, en évitant des pièges, et un terrible monstre, **le Wumpus**. 

Alan se trouve dans une grille de dimension $n\times n$ (où $n$ est un paramètre du problème), il peut chercher à se déplacer vers la gauche, vers la droite, vers le haut, ou vers le bas. Ces actions sont disponibles si elles ne le font pas sortir de la grille et permettent de déplacer Alan. Alan peut aussi jeter un sortilège pour abattre le Wumpus. Cette action est disponible (et réussit) si le monstre se trouve dans une case adjacente. Malheureusement, la grille contient des pièges, et si Alan se déplace sur un piège ou sur le Wumpus, alors il se retrouve capturé dans un état où plus aucune action n'est possible. Utiliser un sort a un coût de 5, et se déplacer un coût de 1.

Le but du jeu est le suivant, Alan doit abattre le Wumpus, puis atteindre le trésor. 

Le Wumpus, le trésor, et les pièges se trouvent dans des cases connues du labyrinthe mais qui dépendent de l'instance considérée. 

In [1]:
import nbimporter
import numpy as np
from abc import ABC, abstractmethod
from IASearch import Problem

class Wumpus(Problem):    
    """
    A class to define a simple Wumpus problem.
    
    
    Attributes
    ----------
    ELEMENTS : dict
        Keys are strings and Values are chars. Used to describe the different elements in the maze
    ACTIONS : dict
        Keys are strings and Values are chars. Used to describe the different possible actions
    n : int
       The maze in an n * n grid
    wumpus_position: (int,int)
       The position of the wumpus in the maze
    treasure_position: (int,int)
       The position of the treasure in the maze
    maze : numpy.array
       An array of char representing the maze
       
    Methods
    -------
    actions(state)
        Yields the set of valid actions
    transition(state, action)
        The transtion function of the search problem
    isFinal(state)
        Asserts if a state is final or not
    actionCost(state, action)
        The cost function of the search problem
    getInitialState()
        Yields the initial state of the search problem
    heuristic(state)
        An heuristic function to guide the search
    """
    
    ELEMENTS = {"EMPTY" : 'E', "TREASURE" : 'T', "SNARE" : 'S', "WUMPUS": 'W'}
    ACTIONS = {"LEFT" : 'L', "RIGHT" : 'R', "UP" : 'U', "DOWN": 'D', "MAGIC": 'M'}
    
    
    def __init__(self):
        self.n = 5
        self.wumpus_position = (2,3)
        self.treasure_position = (2,4)
        self.maze = np.empty((self.n,self.n), np.dtype(str))
        self.maze[:] = Wumpus.ELEMENTS["EMPTY"]
        self.maze[1,3] = Wumpus.ELEMENTS["SNARE"]
        self.maze[3,3] = Wumpus.ELEMENTS["SNARE"]
        self.maze[1,4] = Wumpus.ELEMENTS["SNARE"]
        self.maze[2,1] = Wumpus.ELEMENTS["SNARE"]
        self.maze[self.treasure_position] = Wumpus.ELEMENTS["TREASURE"]
        self.maze[self.wumpus_position] = Wumpus.ELEMENTS["WUMPUS"]
       
    class WumpusState:
        """
        Inner state class to define a state in the Wumpus problem.
        
        Attributes
        ----------
        position : (int, int)
            a position in the maze
        wumpus_beaten : bool
            True if the Wumpus is defeated else False
        """
        
        def __init__(self,pos = (0,0),wb = False):
            """
            Parameters
            ----------
            pos : (int, int), optional
                a position in the maze. The default is (0,0).
            wb : bool, optional
                True if the Wumpus is defeated else False. The default is False.
            """
            self.position = pos 
            self.wumpus_beaten = wb
            
        def __str__(self):
            return str(self.position) + " " + str(self.wumpus_beaten)
        
        def __hash__(self):
            return hash(self.position) + hash(self.wumpus_beaten)
        
        def __eq__(self, other):
            return (self.position == other.position) and self.wumpus_beaten == other.wumpus_beaten
        
        def __lt__(self,other):
            return (self.position,self.wumpus_beaten) < (other.position, other.wumpus_beaten)
        
        __repr__ = __str__
         
    def actions(self, state):
        """ Yields the set of valid actions. The agent may move if it is not trapped by a snare or the Wumpus and may use magic if the Wumpus is close. 
        Parameters
        ----------
        state : State
            a given state

        Returns
        -------
        set
            a set of chars representing the valid actions in the given state     
        """
        
        posx, posy = state.position
        res = set()
        if (self.maze[posx,posy] == Wumpus.ELEMENTS["SNARE"])\
            or ((self.maze[posx,posy] == Wumpus.ELEMENTS["WUMPUS"]) and (state.wumpus_beaten == False)):
            return res
        if(posx != 0):
            res.add(Wumpus.ACTIONS["LEFT"])
        if(posx != self.n-1):
            res.add(Wumpus.ACTIONS["RIGHT"])
        if(posy != 0):
            res.add(Wumpus.ACTIONS["DOWN"])
        if(posy != self.n-1):
            res.add(Wumpus.ACTIONS["UP"])
        if(((abs(posx-self.wumpus_position[0])+abs(posy-self.wumpus_position[1])) == 1) and (state.wumpus_beaten == False)):
            res.add(Wumpus.ACTIONS["MAGIC"])                           
        return res
    
    def transition(self, state, action):
        """ The transtion function of the Wumpus search problem.
        Parameters
        ----------
        state : State 
            a given state  
        action : char
            a valid action in the state

        Returns
        -------
        the state obtained when performing action in state
        """
        
        posx, posy = state.position
        wb = state.wumpus_beaten
        if action == Wumpus.ACTIONS["LEFT"]:
            return self.WumpusState((posx-1,posy),wb)
        elif action == Wumpus.ACTIONS["RIGHT"]:
            return self.WumpusState((posx+1,posy),wb)
        elif action == Wumpus.ACTIONS["DOWN"]:
            return self.WumpusState((posx,posy-1),wb)
        elif action == Wumpus.ACTIONS["UP"]:
            return self.WumpusState((posx,posy+1),wb)
        elif action == Wumpus.ACTIONS["MAGIC"]:
            return self.WumpusState((posx,posy),True)
        raise Exception("Invalid action")
        
    def isFinal(self, state):
        """ Asserts if a state is final or not. A state is final if the position matches the one of the Treasure and if the Wumpus is defeated.

        Parameters
        ----------
        state : State
            a given state

        Returns
        -------
        True if the state is final else False

        """
        
        return state.position == self.treasure_position and state.wumpus_beaten
        
    def actionCost(self, state, action):
        """ The cost function of the Wumpus search problem.
        Parameters
        ----------
        state : State 
            a given state  
        action : char
            a valid action in the state

        Returns
        -------
        the cost (a positive integer) incurred when performing action in state, 1 for a move action and 5 to use magic
        """
        
        return 5 if action == Wumpus.ACTIONS["MAGIC"] else 1
        
    def getInitialState(self):
        """ Yields the initial state of the Wumpus search problem.
        Returns
        -------
        the initial state of the search problem, by default in (0,0) and the Wumpus alive
        """
        
        return self.WumpusState()

**Cadre partiellement observable.** 
On se place désormais dans un cadre partiellement observable dans lequel Alan ne sait plus exactement dans quel état il se trouve. Heureusement, grâce à sa carte du labyrinthe et ses perceptions, Alan peut espérer s'en sortir. Alan peut en effet sentir s'il est à une extrémité du labyrinthe (perceptions *WallOnLeft*, *WallOnRight*, *WallOnDown*, et *WallOnUp*). Il peut également sentir si un piège se trouve sur une case adjacente (perception *SnareClose*}) ou si le Wumpus se trouve sur une case adjacente (perception *WumpusClose*). 

Nous allons maintenir décrire une classe représentant un problème partiellement observable puis définir une classe représentant le problème de la chasse au Wumpus partiellement observable.

In [ ]:
class POProblem(ABC):
    """
    Abstract class to represent a general partially observable search problem
    
    Attributes
    ----------
    problem : Problem
        The physical problem the partially observable problem is built on.
    
    Methods
    -------
    actions(beliefState)
        Returns the valid actions given a belief-state
    prediction(beliefState, action)
        Returns the set of possible future states given a belief-state and an action
    percepts(state)
        Abstract method which should return the percepts that the agent observes in a given state 
    possiblePercepts(beliefState)
        Returns the set of possible percepts that can be observed given a belief-state
    update(states, action)
        Returns the set of possible future states given a belief-state and an action partitioned with respect to the percepts that would be observed
    isFinal(beliefState)
        Returns True if the beliefState is final else False
    actionCost(beliefState, action)
        Returns the cost of performing an action in a given belief-state, i.e., the maximum possible cost.
    getInitialState()
        Abstract method which should return the initial belief-state.
    """
    
    def __init__(self, problem):
        """
        Parameters
        ----------
        problem : Problem
            The physical problem the partially observable problem is built on.

        """
        self.problem = problem
            
    def actions(self, beliefState):
        """ Returns the valid actions given a belief-state
        
        Parameters
        ----------
        beliefState : set
            a beliefState, i.e., a set of states.

        Returns
        -------
        set
           the set of valid actions in the given belief-state.

        """
        #TODO 
    
    def prediction(self, beliefState, action):
        """ Returns the set of possible future states given a belief-state and an action.
        
        Parameters
        ----------
        beliefState : set
            A beliefState, i.e., a set of states.
        action : char
            A valid action given the belief-state.

        Returns
        -------
        set
            The set of possible future states when performing action in beliefState.
        """
        #TODO
    
    @abstractmethod
    def percepts(self, state):
        """ Abstract method which should return the percepts that the agent receive in a given state.

        Parameters
        ----------
        state : State
            A state of the physical search problem.

        Returns
        -------
        set
            A set of percepts observed in the given state. 

        """
        pass
    
    def possiblePercepts(self, beliefState):
        """Returns the set of possible percepts that can be observed given a belief-state.
        
        Parameters
        ----------
        beliefState : set
            A beliefState, i.e., a set of states.

        Returns
        -------
        set
            The set of possible percepts that can be observed given beliefState.
        """
        return {frozenset(self.percepts(state)) for state in beliefState}
    
    def update(self, states, action):
        """ Returns the set of possible future states given a belief-state and an action partitioned with respect to the percepts that would be observed.
        
        Parameters
        ----------
        beliefState : set
            A beliefState, i.e., a set of states.
        action : char
            A valid action given the belief-state.

        Returns
        -------
        dict
            A dictionnary where keys are percepts and values are sets of states compatible with these percepts.

        """
        possibleNextStates = self.prediction(states, action)
        possibleNextPercepts = self.possiblePercepts(possibleNextStates)
        #TODO
        
    def isFinal(self, beliefState):
        """ Returns True if the beliefState is final else False.

        Parameters
        ----------
        beliefState : set
            A beliefState, i.e., a set of states.

        Returns
        -------
        bool
            True if the beliefState is final else False.

        """
        #TODO
    
    def actionCost(self, beliefState, action):
        """ Returns the cost of performing an action in a given belief-state, i.e., the maximum possible cost.
        
        Parameters
        ----------
        beliefState : set
            A beliefState, i.e., a set of states.
        action : char
            A valid action given the belief-state.

        Returns
        -------
        int 
            the cost of performing action in beliefState.

        """
        res = 0
        for state in beliefState:
            res = max(res, self.problem.actionCost(state,action))
        return res
            
    
    @abstractmethod
    def getInitialState(self):
        """ Abstract method which should return the initial belief-state.

        Returns
        -------
        set
            the initial belief-state. 

        """
        pass

Nous allons maintenant définir une classe représentant le problème de **la chasse au Wumpus partiellement observable**. 

In [ ]:
class POWumpus(POProblem):
    """ A class to represent a partially observable Wumpus problem.
    
    Attributes
    ----------
    PERCEPTS : dict
        Keys are strings and Values are chars. Used to described the different possible percepts the agent can receive  
    problem : Wumpus
        A Wumpus physical search problem
    
    Methods
    -------
    percepts(state)
        Returns the percepts that the agent observes in a given state
    getInitialState()
        Returns the initial belief-state
    heuristic(beliefState)
        An heuristic function to guide the search.
    
    """
    PERCEPTS = {"WallOnLeft" : 'L', "WallOnRight" : 'R', "WallOnDown" : 'D',  "WallOnUp" : 'U', "SnareClose" : 'S', "WumpusClose": 'W'}
    
    def __init__(self):
        self.problem = Wumpus()
        
    def percepts(self, state):
        """ Returns the percepts that the agent observes in a given state.

        Parameters
        ----------
        state : State
            A state of the physical Wumpus search problem.

        Returns
        -------
        set
            A set of percepts observed in the given state.
        """
        res = set()
        posx,posy = state.position
        if(posx == 0):
            res.add(POWumpus.PERCEPTS["WallOnLeft"])
        elif(self.problem.maze[posx-1,posy] == Wumpus.ELEMENTS["SNARE"]):
                res.add(POWumpus.PERCEPTS["SnareClose"])
        if(posx == self.problem.n-1):
            res.add(POWumpus.PERCEPTS["WallOnRight"])
        elif(self.problem.maze[posx+1,posy] == Wumpus.ELEMENTS["SNARE"]):
                res.add(POWumpus.PERCEPTS["SnareClose"])
        if(posy == 0):
            res.add(POWumpus.PERCEPTS["WallOnDown"])
        elif(self.problem.maze[posx,posy-1] == Wumpus.ELEMENTS["SNARE"]):
                res.add(POWumpus.PERCEPTS["SnareClose"])
        if(posy==self.problem.n-1):
            res.add(POWumpus.PERCEPTS["WallOnUp"])
        elif(self.problem.maze[posx,posy+1] == Wumpus.ELEMENTS["SNARE"]):
                res.add(POWumpus.PERCEPTS["SnareClose"])
        if(((abs(posx-self.problem.wumpus_position[0])+abs(posy-self.problem.wumpus_position[1])) == 1) and (state.wumpus_beaten == False)):
                res.add(POWumpus.PERCEPTS["WumpusClose"])
        return res
    
    def getInitialState(self):
        """ Returns the initial belief-state.

        Returns
        -------
        set
            The initial belief-state.

        """
        return {Wumpus.WumpusState(pos = (1,1)),Wumpus.WumpusState(pos = (1,2)),Wumpus.WumpusState(pos = (3,2)),Wumpus.WumpusState(pos = (3,1))}

Nous allons maintenant implémenter l'algorithme AndOrSearch.

In [ ]:
class AndOrSearch:
    """ Class to run the AndOrSearch algorithm.
    This class has to be completed.
    """
    
    def __init__(self,po_problem):
        """
        Parameters
        ----------
        po_problem : POProblem
            The partially observable problem on which the algorithm is used.

        Returns
        -------
        Nested lists giving the conditional plan that should be exectuted. The subplans to be used depend on the percepts that are obtained.

        """
        self.po_problem = po_problem
    
    def solve(self):
        """
        Launchs the AND-OR-SEARCH algorithm from the initial state
        """
        return self.orSearch(self.po_problem.getInitialState(), [])

    def orSearch(self, b_state, path):
        """
        Implements the OrSearch method
        """
        #TODO

    def andSearch(self, b_states, path):
        """
        Implements the AndSearch method
        """
        #TODO

Enfin, testons notre algorithme, pour résoudre le problème du Wumpus partiellement observable. 

In [ ]:
problem = POWumpus()
andOrSearch = AndOrSearch(problem)
print(andOrSearch.solve())